## Linear Regression
An estimator is any model that tries to estimate a variable y from another variable(s) x given pairs of data (x<sub>1</sub>,y<sub>1</sub>),(x<sub>2</sub>,y<sub>2</sub>),...,(x<sub>N</sub>,y<sub>N</sub>)

Regression is when the targets (y) are quantities (not cat vs dog but rather price of a house)

An example would be trying to guess how many people will buy icecream from a shop at any day given the temprature on that day. In this case, the target is the number of icecream people will buy, it is an integer and any value (ex: 14 icecreams) doesn't represent a particular class, so it will be represented as a regression problem. In this case, it turns out that x is also a quantity (temprature); however, this is not necessary and is tackled by approaches other than linear regression.

To estimate the price of the icecream from temperature we gather the data on various days from various different places and get the following data.



We can see that a line (could  also be a curve) could fit the data, but how can we find which line exactly.

#### Line Equation Review
A line in 2D can be parametrized using a slope m and a y-intercept b which would result in it having an equation

$$y = mx+b$$

In the example of the icecream, if the temperature at a day was 30, our prediction for the number of icecreams sold would be m\*30 + b, so we simply need to find m and b.

#### Squared Loss
We need some measurement of how good our line is to be able to find the "best" line, so we will measure the difference between our predictions and the correct values from the data.


Then, we will square all the differences and add them up. This will be the value we try to minimize.

#### Note
The normal equation will probably not work. It is left for you to figure it out and let everyone else know, let's see who figures it out first (it's a tricky issue).




In [51]:
import numpy as np
import pandas as pd

In [52]:
class Linear_Regression():
    '''
    Linear Regression model created using only NumPy
    
    Attributes
    ----------
    weights: np.array of floats
        All the parameters of the model (including bias)
    '''
    slope=0
    def __init__(self):
        self.weights = None
        
    def train(self,data_X,data_y):
        '''
        Train the model using the given data
        
        Parameters
        ----------
        data_X: np.array, shape = (N, num_features)
            Features from the data, each row is one data point. Assumes that a column of ones was added to data_X
        data_y: np.array, shape = (N, num_targets)
            The target values to predict, each row contains the targets for one data point
        '''
        ########################## Insert code here ##########################
        slope = np.linalg.pinv(data_X.T@data_X).dot(data_X.T@data_y).T.reshape(-1)
        return slope 
    
    def predict(self,x_to_predict):
        '''
        Predict using the given value as input
        
        Assumes that self.train(.,.) has been called before calling this method
        
        Parameters
        ----------
        x_to_predict: np.array, shape = (M, num_features)
            A given list of inputs to predict targets for, each row is one input. Assumes that a column of ones was added similar to the training data
        
        Returns
        -------
        np.array of floats, shape = (M, num_targets)
            Predicted values for each input
        '''
        ########################## Insert code here ##########################
        global slope
        y=(x_to_predict@ slope)

        return y

### Import the data and remove useless columns

In [42]:
df = pd.read_csv("train.csv")
df.drop(columns=["Id"],inplace=True)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Handle the missing data (NaNs)

In [53]:
df.drop(columns=df.columns[df.isnull().sum().values>200],inplace=True)
df.dropna(inplace=True)
df.isnull().sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

### Replace categorical data (strings) with numerical values

In [54]:
obj_to_replace = df["MSZoning"].dtype

for column in df.columns:
    if df[column].dtype == obj_to_replace:
        uniques = np.unique(df[column].values)
        for idx,item in enumerate(uniques):
            df[column] = df[column].replace(item,idx)
            
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bias
0,11,4,35,1,3,3,0,4,0,0,2,2,4,2,5,3,102,54,3,1,11,13,1,18.0,2,3,2,2,3,3,5,0,5,0,687,485,0,0,1,4,485,0,0,318,0,0,2,0,2,0,2,3,6,1,1,2004.0,0,1,97,4,4,2,73,9,0,0,0,0,0,4,0,8,4,289,1.0
1,0,3,731,1,3,3,0,0,0,16,2,2,0,2,5,3,103,55,1,1,11,13,1,226.0,2,3,2,2,3,2,1,540,5,0,207,560,0,0,1,4,550,0,0,393,0,0,2,0,3,0,2,4,6,1,1,2005.0,1,1,206,4,4,2,0,57,0,0,0,0,0,4,4,6,5,394,1.0
2,10,3,676,1,3,3,0,0,0,12,1,2,2,5,3,3,58,10,1,1,7,8,1,361.0,3,3,1,3,3,3,4,194,5,0,412,331,0,4,1,0,301,337,0,719,0,0,2,0,5,1,3,9,6,0,5,1960.0,2,1,163,4,4,2,0,0,0,0,0,0,0,0,4,8,4,209,1.0
3,0,3,806,1,3,3,0,4,0,12,2,2,0,2,3,5,54,50,3,1,12,14,2,0.0,3,3,1,2,3,3,4,147,5,0,353,210,0,4,1,4,175,0,0,61,0,0,1,0,2,0,3,1,6,0,1,1956.0,2,0,23,4,4,2,162,0,52,0,0,0,0,5,1,8,4,127,1.0
4,0,3,850,1,0,1,0,0,0,22,2,2,0,2,7,3,103,56,3,1,11,13,3,200.0,2,3,2,0,3,1,2,553,5,0,418,662,0,0,1,4,678,0,0,596,1,0,2,0,3,0,2,5,6,1,1,2005.0,0,2,298,4,4,2,101,38,0,0,0,0,0,6,1,8,4,579,1.0


### Add the bias column (column of ones)

In [55]:
df["bias"] = np.ones(df.shape[0])
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bias
0,11,4,35,1,3,3,0,4,0,0,2,2,4,2,5,3,102,54,3,1,11,13,1,18.0,2,3,2,2,3,3,5,0,5,0,687,485,0,0,1,4,485,0,0,318,0,0,2,0,2,0,2,3,6,1,1,2004.0,0,1,97,4,4,2,73,9,0,0,0,0,0,4,0,8,4,289,1.0
1,0,3,731,1,3,3,0,0,0,16,2,2,0,2,5,3,103,55,1,1,11,13,1,226.0,2,3,2,2,3,2,1,540,5,0,207,560,0,0,1,4,550,0,0,393,0,0,2,0,3,0,2,4,6,1,1,2005.0,1,1,206,4,4,2,0,57,0,0,0,0,0,4,4,6,5,394,1.0
2,10,3,676,1,3,3,0,0,0,12,1,2,2,5,3,3,58,10,1,1,7,8,1,361.0,3,3,1,3,3,3,4,194,5,0,412,331,0,4,1,0,301,337,0,719,0,0,2,0,5,1,3,9,6,0,5,1960.0,2,1,163,4,4,2,0,0,0,0,0,0,0,0,4,8,4,209,1.0
3,0,3,806,1,3,3,0,4,0,12,2,2,0,2,3,5,54,50,3,1,12,14,2,0.0,3,3,1,2,3,3,4,147,5,0,353,210,0,4,1,4,175,0,0,61,0,0,1,0,2,0,3,1,6,0,1,1956.0,2,0,23,4,4,2,162,0,52,0,0,0,0,5,1,8,4,127,1.0
4,0,3,850,1,0,1,0,0,0,22,2,2,0,2,7,3,103,56,3,1,11,13,3,200.0,2,3,2,0,3,1,2,553,5,0,418,662,0,0,1,4,678,0,0,596,1,0,2,0,3,0,2,5,6,1,1,2005.0,0,2,298,4,4,2,101,38,0,0,0,0,0,6,1,8,4,579,1.0


### Divide the data into training, testing, X, and y

In [56]:
df = df.sample(frac=1).reset_index(drop=True)
training_df = df[:-100]
val_df = df[-100:]
training_y = training_df["SalePrice"].values
training_X = training_df.drop(columns=["SalePrice"]).values
val_y = val_df["SalePrice"].values
val_X = val_df.drop(columns=["SalePrice"]).values

print(training_X.shape)
print(np.mean(training_y))

(1238, 74)
272.3513731825525


### Train the linear regressor

In [59]:
# Create and fit the model
LR_regressor = Linear_Regression()
coef=LR_regressor.train(training_X,training_y)
print(coef)
# Calculate Mean Absolute Error (Easier to interpret than MSE)
########################## Insert code here ##########################
sum=0.0
for i in range(1238) :
    sum=sum + abs(predicted_home_prices[i]-training_y[i])

print("Mean Absolute error=",sum/1238)

[-6.26643524e-01 -1.39049178e+01  3.50149511e-02  7.44101996e+01
 -2.13621973e+00 -1.41297233e+00 -7.24283849e+01  1.23756372e-01
  2.18041958e+01 -1.64838612e-01 -5.38312206e-01 -9.68939016e+00
 -1.95085722e+00 -1.22874207e-01  2.26098117e+01  1.38229058e+01
  8.26558085e-01  3.35752204e-01 -1.75845288e-01  1.70101494e-02
 -2.80310264e+00  2.14475304e+00  1.05133842e+00 -5.62890741e-03
 -3.93406560e+00 -1.78977162e+00  4.03516005e+00 -5.57899894e+00
  2.86531493e+00 -2.78907888e+00 -6.02766167e-01 -4.15162758e-02
  9.68417309e-01 -2.20688540e-01 -7.03820242e-02  1.65787394e-01
 -8.36124937e+00 -3.19877941e+00 -6.40321565e-01 -9.67165511e-01
  2.95235586e-02  8.79062395e-02 -1.28497448e+00  2.29454083e-01
  1.29549660e+01  5.40953321e+00  8.36804612e+00  4.16440852e-01
 -7.73009200e+00 -7.98782120e+01 -6.41309386e+00  7.76745208e-01
  1.12665781e+01  1.17375997e+01 -1.14780422e+00 -1.54145135e-01
 -4.07828726e+00  1.37579974e+01  6.27887291e-02 -2.53010101e+00
  2.08877295e+00 -9.20498

### Train using the sklearn linear regressor

In [58]:
from sklearn.linear_model import LinearRegression

# Create and fit the model
########################## Insert code here ##########################
reg = LinearRegression()
reg.fit(training_X,training_y)
sklearn_coef=reg.coef_
sklearn_coef[73]=reg.intercept_
print("regression Coef=",reg.coef_)
# Calculate Mean Absolute Error (Easier to interpret than MSE)
########################## Insert code here ##########################
from sklearn.metrics import mean_absolute_error
predicted_home_prices=reg.predict(training_X)
print("Mean absolute error=",mean_absolute_error(training_y, predicted_home_prices))

# Calculate Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt

Mean=mean_squared_error(training_y,predicted_home_prices)
print("root mean squared error =",sqrt(Mean))

regression Coef= [-6.26643523e-01 -1.39049178e+01  3.50149511e-02  7.44101996e+01
 -2.13621973e+00 -1.41297233e+00 -7.24283846e+01  1.23756373e-01
  2.18041958e+01 -1.64838612e-01 -5.38312206e-01 -9.68939017e+00
 -1.95085722e+00 -1.22874207e-01  2.26098117e+01  1.38229058e+01
  8.26558085e-01  3.35752204e-01 -1.75845288e-01  1.70101498e-02
 -2.80310264e+00  2.14475304e+00  1.05133842e+00 -5.62890742e-03
 -3.93406560e+00 -1.78977162e+00  4.03516005e+00 -5.57899894e+00
  2.86531493e+00 -2.78907888e+00 -6.02766167e-01 -4.15162758e-02
  9.68417308e-01 -2.20688540e-01 -7.03820242e-02  1.65787394e-01
 -8.36124937e+00 -3.19877941e+00 -6.40321566e-01 -9.67165510e-01
  2.95235585e-02  8.79062395e-02 -1.28497448e+00  2.29454083e-01
  1.29549660e+01  5.40953320e+00  8.36804612e+00  4.16440853e-01
 -7.73009200e+00 -7.98782120e+01 -6.41309386e+00  7.76745207e-01
  1.12665781e+01  1.17375997e+01 -1.14780422e+00 -1.54145135e-01
 -4.07828726e+00  1.37579974e+01  6.27887291e-02 -2.53010101e+00
  2.0887